In [7]:
from pathlib import Path
import ffmpeg
import yaml
import tqdm
import sys
import shutil
import pandas as pd
import numpy as np
from tempfile import NamedTemporaryFile

from simple_file_checksum import get_checksum
from joblib import Parallel, delayed


def check_duration(dbtsvfile):
    db = pd.read_csv(dbtsvfile, sep="\t")
    # iterate over the rows
    problematiclist = []
    for index, row in db.iterrows():
        vindex = row["v_index"]
        video_name = row["file_name"]
        tmin = row["tmin"]
        tmax = row["tmax"]
        duration = tmax - tmin
        if tmax - tmin < 2.5:
            print(f"Duration of {vindex} {video_name} : {duration} is less than 2.5 seconds")
            problematiclist.append((vindex, video_name))
    return problematiclist

In [8]:
check_duration("/tank/zhangs/AOT_code_repos/video_curation/edited_video_db_final.tsv")

Duration of 118 couple_swimming_under_the_sea (720p).mp4 : 1.8899999999999997 is less than 2.5 seconds
Duration of 340 pexels-christopher-schultz-5928085 (1080p).mp4 : 2.0 is less than 2.5 seconds
Duration of 405 pexels-cottonbro-5659540 (2160p).mp4 : 2.1 is less than 2.5 seconds
Duration of 530 pexels-erik-mclean-5845159 (2160p).mp4 : 2.4999999999999996 is less than 2.5 seconds
Duration of 578 pexels-islam-bayoumy-7120225 (1080p).mp4 : 2.0 is less than 2.5 seconds
Duration of 592 pexels-jaxon-matthew-willis-18447537 (1080p).mp4 : 1.7 is less than 2.5 seconds
Duration of 607 pexels-juancarlos-córdova-5854716 (2160p).mp4 : 2.4 is less than 2.5 seconds
Duration of 608 pexels-juancarlos-córdova-7492693 (Original).mp4 : 2.0 is less than 2.5 seconds
Duration of 619 pexels-kampus-production-6022792.mp4 : 2.0 is less than 2.5 seconds
Duration of 677 pexels-karolina-grabowska-6332681 (Original).mp4 : 2.4999999999999996 is less than 2.5 seconds
Duration of 742 pexels-kelly-lacy-9468762 (2160p).

[(118, 'couple_swimming_under_the_sea (720p).mp4'),
 (340, 'pexels-christopher-schultz-5928085 (1080p).mp4'),
 (405, 'pexels-cottonbro-5659540 (2160p).mp4'),
 (530, 'pexels-erik-mclean-5845159 (2160p).mp4'),
 (578, 'pexels-islam-bayoumy-7120225 (1080p).mp4'),
 (592, 'pexels-jaxon-matthew-willis-18447537 (1080p).mp4'),
 (607, 'pexels-juancarlos-córdova-5854716 (2160p).mp4'),
 (608, 'pexels-juancarlos-córdova-7492693 (Original).mp4'),
 (619, 'pexels-kampus-production-6022792.mp4'),
 (677, 'pexels-karolina-grabowska-6332681 (Original).mp4'),
 (742, 'pexels-kelly-lacy-9468762 (2160p).mp4'),
 (778, 'pexels-kindel-media-8164443.mp4'),
 (938, 'pexels-mikhail-nilov-7829146 (1080p).mp4'),
 (1039, 'pexels-rishe-clicks-15637121 (Original).mp4'),
 (1061, 'pexels-rodnae-productions-5993246 (1080p).mp4'),
 (1318, 'pexels-tima-miroshnichenko-5586521 (2160p).mp4'),
 (1501, 'pexels-юра-9990301 (Original).mp4'),
 (1700, 'production_id_4106919 (2160p).mp4'),
 (1915, 'production_id_4806533 (2160p).mp4'),
